In [43]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output # for graphs and sliders in Dash apps(dcc).
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import os
import plotly.io as pio # for saving plots
from dash import dash_table
import io
import base64 #Encodes and decodes binary data to/from base64, commonly used for embedding files like images.
import tempfile
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

img_dir = "images"

data_path=r"C:\Users\ADMIN\OneDrive\Desktop\FINAL\Accidents.csv"
data = pd.read_csv(data_path)
data.dropna(inplace=True)
data['Hour'] = pd.to_datetime(data['Time'], format='%H:%M', errors='coerce').dt.hour

app = dash.Dash(__name__, suppress_callback_exceptions=True)

app.layout = html.Div([
    html.H1("Traffic Accident Analysis", style={'textAlign': 'center'}),
    html.Div([
       html.Div([
            html.Label("Upload Traffic Accident Data (CSV):"),
            dcc.Upload(
        id='upload-data',
        children=html.Button('Upload File'),
        multiple=False 
        ),
        html.Div(id='output-data-upload', style={'marginTop': '20px', 'fontWeight': 'bold'}),
        ], style={'textAlign': 'center', 'marginTop': '20px'}),

        
        html.Div([
            html.Label("Select Day:"),
            dcc.Dropdown(
                id='day-dropdown',
                options=[{'label': f'Day {day}', 'value': day} for day in data['Day_of_Week'].unique()],
                clearable=False,
                style={'backgroundColor': '#f9f9f9'}
            ),
        ], style={'width': '30%', 'display': 'inline-block'}),

        html.Div([
            html.Label(" Select Region:"),
            dcc.Dropdown(
                id='urban-rural-dropdown',
                options=[{'label': 'Urban', 'value': '0'}, {'label': 'Rural', 'value': '1'}],
                clearable=False,
                style={'backgroundColor': '#f9f9f9'}
            ),
        ], style={'width': '30%', 'display': 'inline-block'}),
    ], style={'display': 'flex', 'justifyContent': 'space-between'}),

    html.H2("Dataset", style={'textAlign': 'center'}),
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in data.columns],
        data=data.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'},
        style_cell={'textAlign': 'left', 'padding': '5px'},
    ),

    dcc.Graph(id='accident-severity-bar'),
    dcc.Graph(id='accidents-by-hour'),
    dcc.Graph(id='correlation-matrix'),
    dcc.Graph(id='weather-vs-road'),
    dcc.Graph(id='severity-vs-speed-limit'),
    dcc.Graph(id='accidents-by-police-attendance'),
    dcc.Graph(id='heatmap'),
    dcc.Graph(id='road-surface-conditions'),
    dcc.Graph(id='weather-conditions'),
    dcc.Graph(id='cluster-map'),
    dcc.Graph(id='hourly-heatmap'),
    dcc.Graph(id='speed-vs-road'),


    html.Div([
        html.Button("Download Report", id="download-button", n_clicks=0, style={'padding': '10px', 'fontSize': '16px', 'backgroundColor': 'lightblue', 'border': 'none', 'cursor': 'pointer'}),
        dcc.Download(id="download-report")
    ], style={'textAlign': 'center', 'marginTop': '20px', 'border': '2px solid black'}),
])


@app.callback(
    [Output('accident-severity-bar', 'figure'),
     Output('accidents-by-hour', 'figure'),
     Output('correlation-matrix', 'figure'),
    Output('weather-vs-road', 'figure'),
     Output('severity-vs-speed-limit', 'figure'),
     Output('accidents-by-police-attendance', 'figure'),
    Output('heatmap', 'figure'),
    Output('weather-conditions', 'figure'),
    Output('road-surface-conditions', 'figure'),
    Output('cluster-map', 'figure'),
    Output('hourly-heatmap', 'figure'),
    Output('speed-vs-road', 'figure'),
    Output('table', 'data'),
     Output('download-report', 'data'),
     Output('output-data-upload', 'children')
    ],


    [Input('day-dropdown', 'value'),
    Input('urban-rural-dropdown', 'value'),
     Input('download-button','n_clicks'),
    Input('upload-data', 'contents')]
)
def update_graphs(day_of_week, urban_or_rural,n_clicks,upload_contents):
    if upload_contents is not None:
            content_type, content_string = upload_contents.split(',')
            decoded = base64.b64decode(content_string)
            uploaded_data = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
            uploaded_data.dropna(inplace=True)
            uploaded_data['Hour'] = pd.to_datetime(uploaded_data['Time'], format='%H:%M', errors='coerce').dt.hour
            filtered_data = uploaded_data
            upload_message = "Data successfully uploaded!" 
#             print("new data")
    else:
            filtered_data = data  
            upload_message = "No data uploaded yet." 




    try:
        # Filtering data for the selected day
        filtered_data1 = data[data['Day_of_Week'] == day_of_week]

        # Ensuring necessary columns are present
        if 'Accident_Severity' not in data.columns or 'Day_of_Week' not in data.columns:
            fig1 = go.Figure().update_layout(
                title="Missing Data: 'Accident_Severity' or 'Day_of_Week' columns are not available",
                xaxis_title="Accident Severity",
                yaxis_title="Count"
            )
        else:
            # Droping rows with missing values in the 'Accident_Severity' column
            filtered_data1 = filtered_data1.dropna(subset=['Accident_Severity'])

            # Handling case where no data exists for the selected day
            if filtered_data1.empty:
                fig1 = go.Figure().update_layout(
                    title=f"No accidents recorded on Day {day_of_week}",
                    xaxis_title="Accident Severity",
                    yaxis_title="Count"
                )
            else:
                
                severity_counts = filtered_data1['Accident_Severity'].value_counts().reset_index()
                severity_counts.columns = ['Accident_Severity', 'Count']

                
                fig1 = px.bar(
                    severity_counts,
                    x='Accident_Severity',
                    y='Count',
                    title=f'Accident Severity on Day {day_of_week}',
                    labels={'Accident_Severity': 'Accident Severity', 'Count': 'Number of Accidents'},
                    #text='Count'  
                )
                fig1.update_traces(textposition='outside')
                fig1.update_layout(
                    yaxis=dict(
                    tickmode='linear',  
                    dtick=1,           
                    title='count'
                    ),
                
                    xaxis=dict(
                    tickmode='linear', 
                    dtick=1        
#                     title='count'
                    ), 
                    bargap=0.2)
                

    except Exception as e:
        fig1 = go.Figure().update_layout(
            title=f"Error: {str(e)}",
            xaxis_title="Accident Severity",
            yaxis_title="Count"
        )





    try:
        filtered_data2 = data[data['Day_of_Week'] == day_of_week]

        # Ensuring the 'Hour' column exists
        if 'Hour' not in filtered_data2.columns:
            fig2= go.Figure().update_layout(
                title="Missing Data: 'Hour' column is not available"
            )

        # Droping rows where the 'Hour' column is NaN or invalid
        filtered_data2 = filtered_data2.dropna(subset=['Hour'])

        # if there's enough data after filtering
        if filtered_data2.empty:
           fig2= go.Figure().update_layout(
                title=f"No accidents data available for Day {day_of_week}",
                xaxis_title="Hour of Day",
                yaxis_title="Accident Count"
            )

       
        fig2 = px.histogram(
            filtered_data2,
            x='Hour',
            nbins=24,  
            title=f'Accidents by Hour on Day {day_of_week}',
            labels={'Hour': 'Hour of Day'}
        )

       
        fig2.update_layout(
            bargap=0.2,
            xaxis_title="Hour of Day",
            yaxis_title="Number of Accidents",
            title=f"Accidents by Hour on Day {day_of_week}"
        )
        img_path = os.path.join(img_dir, f"img2.png")
        


    except Exception as e:
        fig2= go.Figure().update_layout(
            title=f"Error: {str(e)}",
            xaxis_title="Hour of Day",
            yaxis_title="Accident Count"
        )




    try:
        # Selecting  numeric columns for correlation
        numeric_columns = ['Accident_Severity', 'Speed_limit', 'Number_of_Vehicles', 'Number_of_Casualties']

        filtered_data3 = data[(data['Day_of_Week'] == day_of_week) & (data['Urban_or_Rural_Area'] ==int( urban_or_rural))][numeric_columns]

        # Converting columns to numeric 
        for col in numeric_columns:
            filtered_data3[col] = pd.to_numeric(filtered_data3[col], errors='coerce')

        filtered_data3 = filtered_data3.dropna()

        if filtered_data3.empty:
            fig3= go.Figure().update_layout(
                title="No data available for correlation matrix",
                xaxis_title="Variable1",
                yaxis_title="Variable2"
            )

     
        correlation_matrix = filtered_data3.corr()

        fig3 = px.imshow(
            correlation_matrix,
            text_auto=True,
            color_continuous_scale='Viridis',
            title="Correlation Matrix of Key Variables",
            labels={'x': 'Variables', 'y': 'Variables', 'color': 'Correlation'}
        )
        fig3.update_layout(
            xaxis_title="Variable1",
            yaxis_title="Variable2",
            autosize=True
        )
        fig3.update_layout(
                    yaxis=dict(
                    tickmode='linear',  
                    dtick=1           
                    
                    ),
                
                    xaxis=dict(
                    tickmode='linear', 
                    dtick=1          
                   # title='count'
                    ), 
                    bargap=0.2)
        img_path = os.path.join(img_dir, f"img3.png")
       



    except Exception as e:
     
        fig3= go.Figure().update_layout(
            title=f"Error: {str(e)}",
            xaxis_title="Variables",
            yaxis_title="Variables"
        )


    try:
        
        filtered_data4 = data[data['Day_of_Week'] == day_of_week]

        # Ensuring required columns exist
        if 'Road_Surface_Conditions' not in filtered_data4.columns or 'Weather_Conditions' not in filtered_data4.columns:
           fig4= go.Figure().update_layout(title="Missing Data: 'Road_Surface_Conditions' or 'Weather_Conditions' columns are not available")

        # Droping rows with missing values in the relevant columns
        filtered_data4 = filtered_data4.dropna(subset=['Road_Surface_Conditions', 'Weather_Conditions'])

        if filtered_data4.empty:
            fig4=go.Figure().update_layout(title="No data available for the selected day")

        fig4 = px.histogram(
            filtered_data4,
            x='Road_Surface_Conditions',
            color='Weather_Conditions',
            title=f'Road Surface Conditions vs Weather on Day {day_of_week}',
            labels={'Road_Surface_Conditions': 'Road Surface Conditions', 'Weather_Conditions': 'Weather Conditions'}
        )
        fig4.update_traces(textposition='outside')
        fig4.update_layout(
                    yaxis=dict(
                    tickmode='linear',  
                    dtick=1
                    ),
                
                    xaxis=dict(
                    tickmode='linear', 
                    dtick=1          
                   # title='count'
                    ), 
                    bargap=0.2)
        img_path = os.path.join(img_dir, f"img4.png")
        # pio.write_image(fig, img_path)


    except Exception as e:
        fig4= go.Figure().update_layout(
            title=f"Error: {str(e)}",
            xaxis_title="Road Surface Conditions",
            yaxis_title="Count"
        )

    try:

        filtered_data5 = data[data['Day_of_Week'] == day_of_week]

        if 'Speed_limit' not in filtered_data5.columns or 'Accident_Severity' not in filtered_data5.columns:
            fig5=go.Figure().update_layout(
                title="Missing Data: Required columns ('Speed_limit', 'Accident_Severity') not available"
            )

       
        filtered_data5 = filtered_data5.dropna(subset=['Speed_limit', 'Accident_Severity'])

        if filtered_data5.empty:
            fig5= go.Figure().update_layout(
                title=f"No data available for Day {day_of_week}",
                xaxis_title="Accident Severity",
                yaxis_title="Speed Limit"
            )

        fig5 = px.box(
            filtered_data5,
            x='Accident_Severity',
            y='Speed_limit',
            title=f'Speed Limit vs Severity on Day {day_of_week}',
            labels={'Accident_Severity': 'Accident Severity', 'Speed_limit': 'Speed Limit'}
        )

        fig5.update_layout(
            xaxis_title="Accident Severity",
            yaxis_title="Speed Limit",
            title=f"Speed Limit vs Severity on Day {day_of_week}"
        )
        fig5.update_layout(
                    yaxis=dict(
                    tickmode='linear',  
                    dtick=1        
                   
                    ),
                
                    xaxis=dict(
                    tickmode='linear', 
                    dtick=1          
                   # title='count'
                    ), 
                    bargap=0.2)
        img_path = os.path.join(img_dir, f"img5.png")
        # pio.write_image(fig, img_path)



    except Exception as e:
        fig5= go.Figure().update_layout(
            title=f"Error: {str(e)}",
            xaxis_title="Accident Severity",
            yaxis_title="Speed Limit"
        )

    try:
        
        filtered_data6 = data[data['Day_of_Week'] == day_of_week]

        if 'Did_Police_Officer_Attend_Scene_of_Accident' not in filtered_data6.columns or 'Accident_Severity' not in filtered_data6.columns:
            fig6=go.Figure().update_layout(title="Missing Data: 'Did_Police_Officer_Attend_Scene_of_Accident' or 'Accident_Severity' columns are not available")

       
        filtered_data6 = filtered_data6.dropna(subset=['Did_Police_Officer_Attend_Scene_of_Accident', 'Accident_Severity'])

       
        if filtered_data6.empty:
           fig6= go.Figure().update_layout(title="No data available for the selected day")

       
        fig6 = px.histogram(
            filtered_data6,
            x='Did_Police_Officer_Attend_Scene_of_Accident',
            color='Accident_Severity',
            title=f'Police Attendance for Accidents on Day {day_of_week}',
            labels={'Did_Police_Officer_Attend_Scene_of_Accident': 'Police Attendance', 'Accident_Severity': 'Accident Severity'},
            category_orders={'Did_Police_Officer_Attend_Scene_of_Accident': ['Yes', 'No']}  # Assuming 'Yes'/'No' values
        )
        fig6.update_layout(barmode='stack', xaxis_title="Police Attendance", yaxis_title="Count")
        
        fig6.update_traces(textposition='outside')
        fig6.update_layout(
                    yaxis=dict(
                    tickmode='linear',  
                    dtick=1
                    ),
                
                    xaxis=dict(
                    tickmode='linear', 
                    dtick=1          
                   # title='count'
                    ), 
                    bargap=0.2)
        img_path = os.path.join(img_dir, f"img6.png")
        # pio.write_image(fig, img_path)



    except Exception as e:
        fig6= go.Figure().update_layout(
            title=f"Error: {str(e)}",
            xaxis_title="Police Attendance",
            yaxis_title="Accident Count"
        )
    try:
       
        filtered_data7 = data[data['Day_of_Week'] == day_of_week]

        
        required_columns = ['Accident_Index', 'Hour', 'Speed_limit']
        if any(col not in filtered_data7.columns for col in required_columns):
           fig7= go.Figure().update_layout(title="Missing Data: Required columns are missing")

        filtered_data7 = filtered_data7.dropna(subset=required_columns)

      
        if filtered_data7.empty:
           fig7=go.Figure().update_layout(title=f"No accidents on Day {day_of_week}")

       
        pivot_table = pd.pivot_table(
            filtered_data7,
            values='Accident_Index',
            index='Hour',
            columns='Speed_limit',
            aggfunc='count',
            fill_value=0
        )

        if pivot_table.empty:
            fig7= go.Figure().update_layout(title=f"No data available for heatmap on Day {day_of_week}")

       
        fig7 = px.imshow(
            pivot_table,
            labels=dict(x="Speed Limit", y="Hour", color="Accident Count"),
            title="Heatmap: Hour vs Speed Limit"
        )
        fig7.update_layout(
      width=1000, 
      height=500   
      )
        
        fig7.update_layout(
                    yaxis=dict(
                    tickmode='linear',  
                    dtick=1
                    ),
                
                    xaxis=dict(
                    tickmode='linear', 
                    dtick=1          
                   # title='count'
                    ), 
                    bargap=0.2)

        # pio.write_image(fig, img_path)

    except Exception as e:
       
       fig7= go.Figure().update_layout(
            title=f"Error: {str(e)}",
            xaxis_title="Speed Limit",
            yaxis_title="Hour"
        )
    filtered_data8 = data[data['Day_of_Week'] == day_of_week]

       
    if 'Road_Surface_Conditions' not in filtered_data8.columns or 'Accident_Severity' not in filtered_data8.columns:
        fig8= go.Figure().update_layout(
                title="Missing Data: Ensure 'Road_Surface_Conditions' and 'Accident_Severity' are present"
            )

     
    filtered_data8 = filtered_data8.dropna(subset=['Road_Surface_Conditions', 'Accident_Severity'])

       
    if filtered_data8.empty:
            fig8 =go.Figure().update_layout(
                title="No Data Available for Plotting",
                xaxis_title="Road Surface Conditions",
                yaxis_title="Accident Severity"
            )

       
    fig8 = px.histogram(
            filtered_data8,
            x='Road_Surface_Conditions',
            color='Accident_Severity',
            title=f'Road Surface Conditions and Accident Severity on Day {day_of_week}',
            labels={'Road_Surface_Conditions': 'Road Surface Conditions', 'Accident_Severity': 'Accident Severity'}
        )

       
    fig8.update_layout(
            xaxis_title="Road Surface Conditions",
            yaxis_title="Count of Accidents",
            title=f"Road Surface Conditions vs Accident Severity on Day {day_of_week}"
        )
    fig8.update_layout(
                    yaxis=dict(
                    tickmode='linear',  
                    dtick=1
                    ),
                
                    xaxis=dict(
                    tickmode='linear', 
                    dtick=1          
                   # title='count'
                    ), 
                    bargap=0.2)

    
    try:
       
        filtered_data = data[data['Day_of_Week'] == day_of_week]

        if 'Weather_Conditions' not in filtered_data.columns or 'Accident_Severity' not in filtered_data.columns:
            fig9= go.Figure().update_layout(title="Missing Data: 'Weather_Conditions' or 'Accident_Severity' columns are not available")

        filtered_data = filtered_data.dropna(subset=['Weather_Conditions', 'Accident_Severity'])

       
        if filtered_data.empty:
            fig9=go.Figure().update_layout(title="No data available for the selected day")

        
        fig9 = px.histogram(
            filtered_data,
            x='Weather_Conditions',
            color='Accident_Severity',
            title=f'Weather Conditions on Day {day_of_week}',
            labels={'Weather_Conditions': 'Weather Conditions', 'Accident_Severity': 'Accident Severity'},
            category_orders={'Weather_Conditions': filtered_data['Weather_Conditions'].unique()}  # Automatically order by unique weather conditions
        )
        
        fig9.update_layout(barmode='stack', xaxis_title="Weather Conditions", yaxis_title="Count")
        fig9.update_layout(
                    yaxis=dict(
                    tickmode='linear',  
                    dtick=1,           
                    title='Accident_Severity'
                    ),
                
                    xaxis=dict(
                    tickmode='linear', 
                    dtick=1          
                   # title='count'
                    ), 
                    bargap=0.2)
                    
        img_path = os.path.join(img_dir, f"img9.png")
        # pio.write_image(fig, img_path)



    except Exception as e:
        
       fig9= go.Figure().update_layout(
            title=f"Error: {str(e)}",
            xaxis_title="Weather Conditions",
            yaxis_title="Accident Count"
        )

    try:
        
        filtered_data = data[data['Day_of_Week'] == day_of_week]

     
        if 'Latitude' not in filtered_data.columns or 'Longitude' not in filtered_data.columns or 'Speed_limit' not in filtered_data.columns:
            fig10=go.Figure().update_layout(title="Missing Data: Required columns ('Latitude', 'Longitude', 'Speed_limit') not available")

       
        filtered_data = filtered_data.dropna(subset=['Latitude', 'Longitude', 'Speed_limit'])

      
        if filtered_data.shape[0] < 10:
            fig10=go.Figure().update_layout(title="Insufficient data for clustering")

       
        features = filtered_data[['Speed_limit', 'Latitude', 'Longitude']]

        
        scaler = StandardScaler()
        scaled_features = scaler.fit_transform(features)

       
        dbscan = DBSCAN(eps=0.3, min_samples=10)
        clusters = dbscan.fit_predict(scaled_features)

        
        filtered_data['Cluster'] = clusters

       
        fig10 = px.scatter(
            filtered_data,
            x='Longitude',
            y='Latitude',
            color='Cluster',
            title=f'Accident Clusters on Day {day_of_week}',
            labels={'Longitude': 'Longitude', 'Latitude': 'Latitude', 'Cluster': 'Cluster'},
            hover_data=['Speed_limit']
        )

        
        fig10.update_layout(
            xaxis_title="Longitude",
            yaxis_title="Latitude",
            title=f"Accident Clusters on Day {day_of_week}",
            showlegend=True
        )
        fig10.update_layout(
                    yaxis=dict(
                    tickmode='linear',  
                    dtick=1
                    ),
                
                    xaxis=dict(
                    tickmode='linear', 
                    dtick=1          
                   # title='count'
                    ), 
                    bargap=0.2)
        img_path = os.path.join(img_dir, f"img10.png")
        # pio.write_image(fig, img_path)




    except Exception as e:
        fig10= go.Figure().update_layout(
            title=f"Error: {str(e)}",
            xaxis_title="Longitude",
            yaxis_title="Latitude"
        )

    try:
        
        filtered_data = data[data['Day_of_Week'] == day_of_week]

        if 'Hour' not in filtered_data.columns or 'Weather_Conditions' not in filtered_data.columns:
            fig11= go.Figure().update_layout(title="Missing Data: 'Hour' or 'Weather_Conditions' columns are not available")

       
        filtered_data = filtered_data.dropna(subset=['Hour', 'Weather_Conditions'])

        
        if filtered_data.empty:
            fig11= go.Figure().update_layout(title="No data available for the selected day")

       
        pivot_table = pd.pivot_table(
            filtered_data,
            values='Accident_Index', 
            index='Hour',
            columns='Weather_Conditions',
            aggfunc='count',  # Countig number of accidents for each combination
            fill_value=0  
        )

        
        fig11 = px.imshow(
            pivot_table,
            labels=dict(x="Weather Conditions", y="Hour", color="Accident Count"),
            title=f"Hourly Weather Heatmap on Day {day_of_week}",
            color_continuous_scale='Viridis'  
        )
        fig11.update_layout(
        width=1000,  
        height=800   
        )
        fig11.update_layout(
                    yaxis=dict(
                    tickmode='linear',  
                    dtick=1
                    ),
                
                    xaxis=dict(
                    tickmode='linear', 
                    dtick=1          
                   # title='count'
                    ), 
                    bargap=0.2)
        img_path = os.path.join(img_dir, f"img11.png")
        # pio.write_image(fig, img_path)


    except Exception as e:
        
        fig11=go.Figure().update_layout(
            title=f"Error: {str(e)}",
            xaxis_title="Weather Conditions",
            yaxis_title="Hour"
        )

    try:
        
        filtered_data = data[(data['Day_of_Week'] == day_of_week) & (data['Urban_or_Rural_Area'] ==int( urban_or_rural))]

        if 'Speed_limit' not in filtered_data.columns or 'Road_Type' not in filtered_data.columns or 'Accident_Index' not in filtered_data.columns:
            fig12= go.Figure().update_layout(title="Missing Data: 'Speed_limit', 'Road_Type', or 'Accident_Index' columns are not available")

       
        filtered_data = filtered_data.dropna(subset=['Speed_limit', 'Road_Type', 'Accident_Index'])

       
        if filtered_data.empty:
            fig12= go.Figure().update_layout(title="No data available for the selected day")

       
        pivot_table = pd.pivot_table(
            filtered_data,
            values='Accident_Index',  
            index='Speed_limit',
            columns='Road_Type',
            aggfunc='count', 
            fill_value=0  
        )

       
        fig12 = px.bar(
            pivot_table,
            barmode='stack',  # Stack bars for showing the total count of accidents for each speed limit
            title=f"Speed Limit vs Road Type on Day {day_of_week}",
            labels={"index": "Speed Limit", "value": "Accident Count"},
            height=400
        )
        
        fig12.update_layout(
                    yaxis=dict(
                    tickmode='linear',  
                    dtick=1,           
                    title='Accident_Severity'
                    ),
                
                    xaxis=dict(
                    tickmode='linear', 
                    dtick=1          
                   # title='count'
                    ), 
                    bargap=0.2)


    except Exception as e:
       
        fig12= go.Figure().update_layout(
            title=f"Error: {str(e)}",
            xaxis_title="Speed Limit",
            yaxis_title="Accident Count"
        )



    if n_clicks > 0:
       
        pdf_buffer = io.BytesIO()
        c = canvas.Canvas(pdf_buffer, pagesize=letter)
        c.setFont("Helvetica-Bold", 16)
    #     c.showPage()
        c.drawString(200, 750, f"Traffic Accident Analysis Report")

        y_position = 400
        figures = [ fig1,fig2,fig3, fig4, fig5, fig6, fig7, fig8, fig9, fig10, fig11, fig12]
        for fig in figures:
            # Skiping empty figures
            if fig is None:
                continue

            # Converting figure to image (PNG)
            img_bytes = io.BytesIO()
            pio.write_image(fig, img_bytes, format='png')
            img_bytes.seek(0)  

            # Saving the image to a temporary file
            img_file = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
            img_file.write(img_bytes.read())
            img_file.close()

            # Drawing the image onto the PDF
            c.drawImage(img_file.name, 100, y_position, width=400, height=300)

            # Adjusting the y_position for the next image
            y_position -= 310  

            # If there is not enough space on the current page, create a new page
            if y_position < 100:
                c.showPage()
                y_position = 500  # Reseting Y position for the new page

        
        c.save()

        pdf_buffer.seek(0)
        return (fig1, fig2,fig3,fig4,fig5,fig6,fig7,fig8,fig9,fig10,fig11,fig12,  # Return the figures (remaining ones as placeholders)
                filtered_data1.to_dict('records'),  # Return table data
                dcc.send_bytes(pdf_buffer.getvalue(), "Accident_Report.pdf"),upload_message)  # Return PDF file


    return[fig1,fig2,fig3,fig4,fig5,fig6,fig7,fig8,fig9,fig10,fig11,fig12,filtered_data.to_dict("records"),None,upload_message]







if __name__ == '__main__':
    app.run_server(debug=True,port=5174)


In [45]:
import folium
from folium.plugins import HeatMap
if len(data) == 0:
    raise ValueError("The dataset is empty.")

sample_size = min(1000, len(data))
data_sample = data.sample(sample_size, replace=False)

map_accidents = folium.Map(location=[data['Latitude'].mean(), data['Longitude'].mean()], zoom_start=10)

heat_data = [[row['Latitude'], row['Longitude']] for index, row in data_sample.iterrows()]
HeatMap(heat_data).add_to(map_accidents)


map_accidents
